In [1]:
from typing import Optional, Callable
from dataclasses import dataclass
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

from functools import partial

np.random.seed(42)

In [2]:
dataset = pd.read_parquet("products_with_names.parquet")

In [3]:
dataset.head()

,product_id,name
0,4036767,"Модуль сменный фильтрующий Аквафор КН, 208731"
1,4050873,"Водоочиститель Аквафор модель Кристалл Н, 2059..."
2,4226160,Развиваем мышление (2-3 года) | Земцова Ольга
3,4644911,Lacoste Вода парфюмерная Pour Femme 50 мл
4,4788809,Сменные Кассеты Для Мужской Бритвы Gillette Ma...


In [4]:
dataset.shape

(238443, 2)

In [5]:
documents_dict = {
    doc[1]["product_id"]: doc[1]["name"] for doc in dataset.iterrows()
}

## Подготовим документы

In [6]:
@dataclass
class Document:
    doc_id: int
    name: str


In [7]:
documents = [Document(doc_id=doc[1]["product_id"], name=doc[1]["name"]) for doc in dataset.iterrows()]

In [8]:
documents[:3]

[Document(doc_id=4036767, name='Модуль сменный фильтрующий Аквафор КН, 208731'),
 Document(doc_id=4050873, name='Водоочиститель Аквафор модель Кристалл Н, 205963 //с краном'),
 Document(doc_id=4226160, name='Развиваем мышление (2-3 года) | Земцова Ольга')]

## Метрики качества

### В этом файле собраны взаимодействия пользователей с товарами в поиске
### Будем считать товары, с которыми провзаимодействовали - позитивами, остальные - негативами

In [9]:
query_product_interactions = pd.read_parquet("query_product_interactions.parquet")

In [10]:
query_product_interactions.sample(5)

,search_query,products
255754,лимон и лайм,"[[688372347, 2]]"
175947,печень индюшиная охлажденная,"[[142120793, 1]]"
160581,озон фреш нашгетсы,"[[1394225566, 1], [1394225559, 1]]"
199324,мом,"[[146944254, 1], [146936389, 2], [148170218, 2..."
104727,зарядное устройство кабель,"[[776057178, 1]]"


In [11]:
query_positives = {
    row[1]["search_query"]: set([x[0] for x in row[1]["products"]]) 
    for row in query_product_interactions.iterrows()
}

In [12]:
query_positives["homecat"]

{34629202,
 34629203,
 139864790,
 139864791,
 139864792,
 177668198,
 177668199,
 215201915,
 248843140,
 257675430,
 352414492,
 667810552}

In [13]:
good_enough_query_positives = {
    k: v
    for k, v in query_positives.items()
    if len(v) > 10
}

In [14]:
list(good_enough_query_positives.keys())[:100]

['durr сыр',
 'fler alpin',
 'huggies трусики 6',
 'monge для кошек',
 'synergetic мыло',
 'алкогольные напитки',
 'альметте сыр',
 'без глютена хлеб',
 'белковые батончики без сахара',
 'болоньезе',
 'бри сыр',
 'вафли',
 'вегета приправа',
 'веник',
 'вермишель barilla',
 'вино',
 'вискас для кошек сухой',
 'вода святой источник',
 'вода снежинская',
 'вода со вкусом',
 'вода фрутоняня',
 'гель для стрики',
 'голень курицы',
 'грудинка сырокопченая',
 'дистья салата',
 'для бассейна',
 'жидкость для полоскания рта',
 'запеканка',
 'зарядка type с',
 'зубочистки',
 'йогурт клубника',
 'карртошка',
 'каша безмолочная детская fleur alpine',
 'каша молочная агуша',
 'каша молочная детская',
 'каши фрутоняня',
 'кекс с изюмом',
 'кета слабосоленая',
 'кето',
 'кола добрый 2 л',
 'колбаски гриль',
 'конверт подарочный',
 'консервация',
 'конфеиы',
 'копченый сыр',
 'краьовые палочки',
 'ктндер',
 'лего майнкрафт',
 'лимонад добрый',
 'майнкрафт',
 'макроны',
 'масло 82,5 сливочное',
 'масл

In [15]:
validation_queries_set = np.random.choice(
    list(good_enough_query_positives.keys()), size=100, replace=False
).tolist()

In [16]:
validation_queries_set

['хлеб нарезной белый',
 'usb lightning',
 'гель для душа женский 750мл',
 'хлеб чесночный',
 'безалкогольное',
 'лезвия для бритвы',
 'шоколадное печенье',
 'мусорные мешки',
 'вода 19л',
 'прокладки урологические для женщин',
 'ессентуки 4 минеральная вода',
 'злаковые конфеты',
 'помидорв',
 'сыр копченый',
 'donat вода',
 'блоки для унитаза',
 'греческий йогурт teos',
 'майонез heinz',
 'кофе капсульный dolce gusto',
 'молоко село зеленое',
 'шампунь estel',
 'рагу свиное мираторг',
 'вертикальный пылесос',
 'агуша вода',
 'для творчества',
 'детский порошок стиральный',
 'иамло оливковое',
 'озон фреш мороженое',
 'без сахара кола',
 'жидкое мыло synergetic',
 'корм жидкий для кошек',
 'каша фрутоняня жидкая',
 'бадминтон набор',
 'joonies xxl',
 'бытовая химия',
 'подушка для путешествий',
 'корнер',
 'эвервес',
 'proplan для собак',
 'томат',
 'кашка фрутоняня',
 'яблоки фуджи',
 'чехол на айфон 14',
 'салат романо свежий',
 'няня фруто',
 'велком',
 'для посуды средство мытья',

In [17]:
validation_query_positives = {
    k: v
    for k, v in query_positives.items()
    if k in validation_queries_set
}

In [18]:
validation_query_positives

{'хлеб чесночный': {148234009,
  148590542,
  175341662,
  266528732,
  321861482,
  326365538,
  346528161,
  396633134,
  398644622,
  566943599,
  719236341,
  871445282,
  1032351727,
  1032351797,
  1061274470,
  1141489583,
  1147086672,
  1149940627,
  1149960068,
  1152411084,
  1214992426,
  1603713078},
 'грасс': {168316689,
  207862018,
  207862019,
  207862120,
  215542718,
  216090127,
  250450073,
  289657309,
  289857440,
  289857826,
  341751956,
  349695676,
  349699275,
  370718753,
  422323059,
  553853260,
  661424231,
  862276595,
  1621042273,
  1631010650},
 'donat вода': {138898172,
  142401068,
  143484936,
  145026870,
  148001285,
  161571260,
  220188988,
  308031476,
  308031749,
  549222934,
  803032431,
  895258653,
  1175035709,
  1292066577,
  1526948555},
 'каша фрутоняня жидкая': {141734954,
  141822580,
  141822582,
  141822583,
  141822587,
  144967330,
  145026870,
  146805007,
  146805009,
  146805010,
  149989767,
  154599313,
  154599314,
  1545

### Точность (precision) и Полнота (recall)

$$ precision = {relevant\_retrieved \over retrieved},\ recall = {relevant\_retrieved \over all\_relevant} $$

In [19]:
@dataclass
class Metrics:
    precision: float
    recall: float
    f1_score: float
        
    def __repr__(self):
        return f"precision = {self.precision}\nrecall = {self.recall}\nf1_score = {self.f1_score}"


In [20]:
def calculate_metrics(ground_truth_set: set[int], search_results_set: set[int]):
    
    # True positives: items that are both in ground truth and search results
    tp = len(ground_truth_set.intersection(search_results_set))
    
    # Precision: tp / (tp + fp)
    precision = tp / len(search_results_set) if len(search_results_set) > 0 else 0.0
    
    # Recall: tp / (tp + fn)
    recall = tp / len(ground_truth_set) if len(ground_truth_set) > 0 else 0.0
    
    # F1-score: harmonic mean of precision and recall
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return Metrics(precision=precision, recall=recall, f1_score=f1_score)


In [21]:
def calculate_validation_metrics(search_function: Callable, limit: int = 20):
    metrics = []
    search_results = {}
    for query, positives in validation_query_positives.items():
        search_results[query] = search_function(query=query, limit=limit)
        metrics.append(
            calculate_metrics(positives, [x.doc_id for x in search_results[query]])
        )
    
    return Metrics(
        precision=np.mean([x.precision for x in metrics]),
        recall=np.mean([x.recall for x in metrics]),
        f1_score=np.mean([x.f1_score for x in metrics]),
    )
    

## Случайный поиск

In [22]:
def search_random(documents: list[Document], query: str, limit: int = 10):
    return np.random.choice(documents, size=limit).tolist()

In [23]:
random_search_results = search_random(documents, "молоко", limit=20)
random_search_results

[Document(doc_id=24962121, name='Бумажные полотенца Zewa Premium Декор, 4 рулона'),
 Document(doc_id=1563909748, name='Vichy Dercos Densi-Solutions Cыворотка для роста, объема и густоты волос со стемоксидином, ресвератролом и рамнозой, 100 мл. Уцененный товар'),
 Document(doc_id=205154973, name='FitnesShock Протеиновое печенье без сахара Crispy Кокос-гречка, набор 12 шт'),
 Document(doc_id=401708866, name='Отривин Бэби Комфорт насадки сменные для детского назального аспиратора, одноразовые и экстрамягкие, 10 шт.'),
 Document(doc_id=584788724, name='Хумус Ozon fresh, с перчиком халапеньо, 110 г'),
 Document(doc_id=1467225146, name='Наполнитель Древесный PET PRIDE Впитывающий Без отдушки 9000г. Уцененный товар'),
 Document(doc_id=1487842550, name='Порошок стиральный Автомат Tide Color 60 стирок 9 кг. Уцененный товар'),
 Document(doc_id=1574052493, name='Детские одноразовые пеленки YokoSun 20 шт, размер 60*90 (10 шт* 2 уп). Уцененный товар'),
 Document(doc_id=1639893650, name='Сухой корм 

In [24]:
calculate_validation_metrics(partial(search_random, documents))

precision = 0.0005
recall = 0.0002702702702702703
f1_score = 0.0003508771929824561

## Простейший поиск

In [25]:
def search_dummy(documents: list[Document], query: str, limit: int = 10):
    return [doc for doc in documents if query in doc.name][:limit]

[doc for doc in search_dummy(documents, 'яблоко', limit=5)]

[Document(doc_id=141141355, name='Matti мюсли с орехом и яблоком, 250 г'),
 Document(doc_id=147218137, name='Сок Добрый, яблоко персик, 0,2 л х 27'),
 Document(doc_id=147677673, name='Компот детский Агуша, яблоко, черноплодная рябина, клубника, 0,2 л'),
 Document(doc_id=225491326, name='Конфеты Ирис SOLENTO, без глютена, Ассорти двухцветный, яблоко, клубника, манго, арбуз, 250г'),
 Document(doc_id=250287549, name='Сок Сады Придонья Exclusive Зелёное яблоко без сахара, 1 л')]

In [26]:
calculate_validation_metrics(partial(search_dummy, documents), limit=20)

precision = 0.0075
recall = 0.004725482906149134
f1_score = 0.005338843038887443

### Проблемы:
- учет регистра
- пунктуация
- словоформы
- стоп-слова
- опечатки

In [27]:
search_dummy(documents, 'Яблоко', limit=2)

[Document(doc_id=140676424, name='Пюре детское Fleur Alpine Яблоко, с 4 месяцев, 90 г'),
 Document(doc_id=142625167, name='Бабушкино Лукошко Кабачок Яблоко пюре с 5 месяцев, 100 г')]

In [28]:
search_dummy(documents, 'яблоко,', limit=2)

[Document(doc_id=147677673, name='Компот детский Агуша, яблоко, черноплодная рябина, клубника, 0,2 л'),
 Document(doc_id=225491326, name='Конфеты Ирис SOLENTO, без глютена, Ассорти двухцветный, яблоко, клубника, манго, арбуз, 250г')]

In [29]:
search_dummy(documents, 'яблоки', limit=2)

[Document(doc_id=1554768232, name='Российские яблоки Агроном-сад, отборные, 4 шт'),
 Document(doc_id=420836847, name='Набор: капуста брокколи, цветная капуста, тыква, яблоки быстрозамороженные THE LEATLES для приготовления детского питания')]

In [30]:
search_dummy(documents, 'для', limit=2)

[Document(doc_id=4788809, name='Сменные Кассеты Для Мужской Бритвы Gillette Mach3, с 3 лезвиями, прочнее, чем сталь, для точного бритья, 2 шт'),
 Document(doc_id=4789070, name='Gillette Fusion5 мужская бритва, 2 кассеты, с 5 лезвиями, уменьшающими трение, с точным триммером для бороды и усов')]

In [31]:
search_dummy(documents, 'яблако', limit=2)

[]

# Пайплайн представления документов:
1. Обработка текста
2. Индексация

## 1. Обработка текста

In [32]:
sample_documents = list(dataset.loc[:5, "name"])
sample_documents

['Модуль сменный фильтрующий Аквафор КН, 208731',
 'Водоочиститель Аквафор модель Кристалл Н, 205963 //с краном',
 'Развиваем мышление (2-3 года) | Земцова Ольга',
 'Lacoste Вода парфюмерная Pour Femme 50 мл',
 'Сменные Кассеты Для Мужской Бритвы Gillette Mach3, с 3 лезвиями, прочнее, чем сталь, для точного бритья, 2 шт',
 'Gillette Fusion5 мужская бритва, 2 кассеты, с 5 лезвиями, уменьшающими трение, с точным триммером для бороды и усов']

### Этапы
- обработка текста
- лингвистический анализ

## 1.1 Обработка текста:
- приведение к нижнему регистру
- обработка от знаков препинания
- токенизация
- избавление от стоп-слов

### 1.1.1 Приведение к нижнему регистру

In [33]:
def lowercase_text(text: str) -> str:
    return text.lower()

In [34]:
documents_lowercased = [lowercase_text(doc) for doc in sample_documents]
documents_lowercased

['модуль сменный фильтрующий аквафор кн, 208731',
 'водоочиститель аквафор модель кристалл н, 205963 //с краном',
 'развиваем мышление (2-3 года) | земцова ольга',
 'lacoste вода парфюмерная pour femme 50 мл',
 'сменные кассеты для мужской бритвы gillette mach3, с 3 лезвиями, прочнее, чем сталь, для точного бритья, 2 шт',
 'gillette fusion5 мужская бритва, 2 кассеты, с 5 лезвиями, уменьшающими трение, с точным триммером для бороды и усов']

### 1.1.2 Замена символов на эквивалентные

In [35]:
symbols_to_replace = {"ё": "е"}

def replace_symbols(text: str, symbols_to_replace: dict[str, str]) -> str:
    for old, new in symbols_to_replace.items():
        text = text.replace(old, new)
    return text

In [36]:
documents_replaced = [lowercase_text(doc) for doc in documents_lowercased]
documents_replaced

['модуль сменный фильтрующий аквафор кн, 208731',
 'водоочиститель аквафор модель кристалл н, 205963 //с краном',
 'развиваем мышление (2-3 года) | земцова ольга',
 'lacoste вода парфюмерная pour femme 50 мл',
 'сменные кассеты для мужской бритвы gillette mach3, с 3 лезвиями, прочнее, чем сталь, для точного бритья, 2 шт',
 'gillette fusion5 мужская бритва, 2 кассеты, с 5 лезвиями, уменьшающими трение, с точным триммером для бороды и усов']

### 1.1.3 Избавление от знаков препинания

In [37]:
import string

In [38]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [39]:
def process_punctuation_simple(text: str) -> str:
    translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text_without_punc = text.translate(translation_table)
    text_without_double_spaces = ' '.join(text_without_punc.split())
    return text_without_double_spaces

In [40]:
process_punctuation_simple("молоко пастеризованное, 2.5 л")

'молоко пастеризованное 2 5 л'

In [41]:
documents_processed_punct = [
    process_punctuation_simple(doc) for doc in documents_replaced
]
documents_processed_punct

['модуль сменный фильтрующий аквафор кн 208731',
 'водоочиститель аквафор модель кристалл н 205963 с краном',
 'развиваем мышление 2 3 года земцова ольга',
 'lacoste вода парфюмерная pour femme 50 мл',
 'сменные кассеты для мужской бритвы gillette mach3 с 3 лезвиями прочнее чем сталь для точного бритья 2 шт',
 'gillette fusion5 мужская бритва 2 кассеты с 5 лезвиями уменьшающими трение с точным триммером для бороды и усов']

In [ ]:
def process_punctuation_smart(text: str) -> str:
    #TODO: придумать более хорошую обработку знаков препинания
    pass

### 1.1.4 Токенизация

In [42]:
def tokenize_simple(text: str) -> list[str]:
    return text.split()

In [43]:
documents_tokenized = [tokenize_simple(doc) for doc in documents_processed_punct]
documents_tokenized

[['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', '208731'],
 ['водоочиститель',
  'аквафор',
  'модель',
  'кристалл',
  'н',
  '205963',
  'с',
  'краном'],
 ['развиваем', 'мышление', '2', '3', 'года', 'земцова', 'ольга'],
 ['lacoste', 'вода', 'парфюмерная', 'pour', 'femme', '50', 'мл'],
 ['сменные',
  'кассеты',
  'для',
  'мужской',
  'бритвы',
  'gillette',
  'mach3',
  'с',
  '3',
  'лезвиями',
  'прочнее',
  'чем',
  'сталь',
  'для',
  'точного',
  'бритья',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужская',
  'бритва',
  '2',
  'кассеты',
  'с',
  '5',
  'лезвиями',
  'уменьшающими',
  'трение',
  'с',
  'точным',
  'триммером',
  'для',
  'бороды',
  'и',
  'усов']]

#### Альтернатива - обработка текста с пунктуацией:

In [44]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/dandrosov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/dandrosov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [45]:
from nltk.tokenize import word_tokenize

def tokenize_with_punct(text: str) -> list[str]:
    return word_tokenize(text, language="russian")

In [46]:
tokenize_with_punct(documents_replaced[2])

['развиваем', 'мышление', '(', '2-3', 'года', ')', '|', 'земцова', 'ольга']

### 1.1.5 Удаление стоп-слов

In [47]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dandrosov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
from nltk.corpus import stopwords
russian_stopwords = set(stopwords.words("russian"))

In [49]:
russian_stopwords

{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [50]:
def remove_stopwords_from_doc(doc: list[str], stopwords: set[str]) -> list[str]:
    return [token for token in doc if token not in stopwords]

In [51]:
documents_processed = [remove_stopwords_from_doc(doc, russian_stopwords) for doc in documents_tokenized]
documents_processed

[['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', '208731'],
 ['водоочиститель', 'аквафор', 'модель', 'кристалл', 'н', '205963', 'краном'],
 ['развиваем', 'мышление', '2', '3', 'года', 'земцова', 'ольга'],
 ['lacoste', 'вода', 'парфюмерная', 'pour', 'femme', '50', 'мл'],
 ['сменные',
  'кассеты',
  'мужской',
  'бритвы',
  'gillette',
  'mach3',
  '3',
  'лезвиями',
  'прочнее',
  'сталь',
  'точного',
  'бритья',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужская',
  'бритва',
  '2',
  'кассеты',
  '5',
  'лезвиями',
  'уменьшающими',
  'трение',
  'точным',
  'триммером',
  'бороды',
  'усов']]

## 1.2 Лингвистический анализ

### 1.2.1 Стемминг

#### Для английского языка

In [52]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
text = "The stemmed form of leaves is leaf"
tokens = word_tokenize(text)
stemmed_words = [stemmer.stem(word) for word in tokens]
stemmed_words

['the', 'stem', 'form', 'of', 'leav', 'is', 'leaf']

#### Для русского языка

In [53]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")

def stem(token: str, stemmer: SnowballStemmer) -> str:
    return stemmer.stem(token)


In [54]:
documents_processed

[['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', '208731'],
 ['водоочиститель', 'аквафор', 'модель', 'кристалл', 'н', '205963', 'краном'],
 ['развиваем', 'мышление', '2', '3', 'года', 'земцова', 'ольга'],
 ['lacoste', 'вода', 'парфюмерная', 'pour', 'femme', '50', 'мл'],
 ['сменные',
  'кассеты',
  'мужской',
  'бритвы',
  'gillette',
  'mach3',
  '3',
  'лезвиями',
  'прочнее',
  'сталь',
  'точного',
  'бритья',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужская',
  'бритва',
  '2',
  'кассеты',
  '5',
  'лезвиями',
  'уменьшающими',
  'трение',
  'точным',
  'триммером',
  'бороды',
  'усов']]

In [55]:
documents_stemmed = [[stem(token, stemmer) for token in doc] for doc in documents_processed]
documents_stemmed

[['модул', 'смен', 'фильтр', 'аквафор', 'кн', '208731'],
 ['водоочистител', 'аквафор', 'модел', 'кристалл', 'н', '205963', 'кран'],
 ['развива', 'мышлен', '2', '3', 'год', 'земцов', 'ольг'],
 ['lacoste', 'вод', 'парфюмерн', 'pour', 'femme', '50', 'мл'],
 ['смен',
  'кассет',
  'мужск',
  'бритв',
  'gillette',
  'mach3',
  '3',
  'лезв',
  'прочн',
  'стал',
  'точн',
  'брит',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужск',
  'бритв',
  '2',
  'кассет',
  '5',
  'лезв',
  'уменьша',
  'трен',
  'точн',
  'триммер',
  'бород',
  'ус']]

### 1.2.2 Лемматизация

#### Для английского языка

In [56]:
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/dandrosov/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dandrosov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [57]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [58]:
text = "The lemmatized form of leaves is leaf"
tokens = word_tokenize(text)
lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
print(lemmatized_words)

['The', 'lemmatized', 'form', 'of', 'leaf', 'is', 'leaf']


#### Для русского языка

In [59]:
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

In [65]:
morph.parse("лабуба")

[Parse(word='лабуба', tag=OpencorporaTag('NOUN,anim,masc,Name sing,nomn'), normal_form='лабуба', score=0.5, methods_stack=((DictionaryAnalyzer(), 'буба', 32, 0), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'ла'))),
 Parse(word='лабуба', tag=OpencorporaTag('NOUN,inan,femn,Sgtm,Geox sing,nomn'), normal_form='лабуба', score=0.5, methods_stack=((DictionaryAnalyzer(), 'уба', 36, 0), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'лаб')))]

In [61]:
def lemmatize_token(token: str, lemmatizer: MorphAnalyzer) -> str:
    return lemmatizer.normal_forms(token)[0]

def lemmatize_document(doc: list[str], lemmatizer: MorphAnalyzer) -> list[str]:
    return [lemmatize_token(token, lemmatizer) for token in doc]

In [62]:
documents_processed

[['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', '208731'],
 ['водоочиститель', 'аквафор', 'модель', 'кристалл', 'н', '205963', 'краном'],
 ['развиваем', 'мышление', '2', '3', 'года', 'земцова', 'ольга'],
 ['lacoste', 'вода', 'парфюмерная', 'pour', 'femme', '50', 'мл'],
 ['сменные',
  'кассеты',
  'мужской',
  'бритвы',
  'gillette',
  'mach3',
  '3',
  'лезвиями',
  'прочнее',
  'сталь',
  'точного',
  'бритья',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужская',
  'бритва',
  '2',
  'кассеты',
  '5',
  'лезвиями',
  'уменьшающими',
  'трение',
  'точным',
  'триммером',
  'бороды',
  'усов']]

In [63]:
documents_lemmatized = [lemmatize_document(doc, morph) for doc in documents_processed]
documents_lemmatized

[['модуль', 'сменный', 'фильтровать', 'аквафора', 'кн', '208731'],
 ['водоочиститель', 'аквафора', 'модель', 'кристалл', 'н', '205963', 'кран'],
 ['развивать', 'мышление', '2', '3', 'год', 'земцов', 'ольга'],
 ['lacoste', 'вода', 'парфюмерный', 'pour', 'femme', '50', 'мл'],
 ['сменный',
  'кассета',
  'мужской',
  'бритва',
  'gillette',
  'mach3',
  '3',
  'лезвие',
  'прочный',
  'сталь',
  'точный',
  'бритьё',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужской',
  'бритва',
  '2',
  'кассета',
  '5',
  'лезвие',
  'уменьшать',
  'трение',
  'точный',
  'триммер',
  'борода',
  'усов']]

## 1.3 Собираем всё вместе

In [66]:
def process_document(
    doc: str,
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer,
) -> list[str]:
    doc = lowercase_text(doc)
    doc = replace_symbols(doc, symbols_to_replace)
    doc = process_punctuation_simple(doc)
    doc_tokens = tokenize_simple(doc)
    doc_tokens = remove_stopwords_from_doc(doc_tokens, russian_stopwords)
    return lemmatize_document(doc_tokens, lemmatizer)

def process_documents(
    docs: list[Document],
    symbols_to_replace: dict[str, str], 
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer,
    progress_bar: bool = True
) -> list[tuple[int, list[str]]]:
    iterator = docs
    if progress_bar:
        iterator = tqdm(docs)
    
    return [
        (doc.doc_id, process_document(doc.name, symbols_to_replace, russian_stopwords, lemmatizer))
        for doc in iterator
    ]


In [67]:
documents[1000]

Document(doc_id=166282743, name='Кухонная вытяжка Weissgauff Aura 850 BL, черный')

In [68]:
process_document(documents[1000].name, symbols_to_replace, russian_stopwords, morph)

['кухонный', 'вытяжка', 'weissgauff', 'aura', '850', 'bl', 'чёрный']

In [69]:
documents_processed = process_documents(documents, symbols_to_replace, russian_stopwords, morph)

  4%|██████▏                                                                                                                                                                 | 8768/238443 [00:04<01:51, 2068.77it/s]


KeyboardInterrupt: 

In [ ]:
with open("documents_processed.json", "w") as f:
    json.dump(documents_processed, f)


In [70]:
with open("documents_processed.json", "r") as f:
    documents_processed = json.load(f)

In [71]:
documents_processed[0]

[4036767, ['модуль', 'сменный', 'фильтровать', 'аквафора', 'кн', '208731']]

## 2. Формирование индекса

In [73]:
sample_documents_processed = documents_processed[:10]
sample_documents_processed

[[4036767, ['модуль', 'сменный', 'фильтровать', 'аквафора', 'кн', '208731']],
 [4050873,
  ['водоочиститель', 'аквафора', 'модель', 'кристалл', 'н', '205963', 'кран']],
 [4226160, ['развивать', 'мышление', '2', '3', 'год', 'земцов', 'ольга']],
 [4644911, ['lacoste', 'вода', 'парфюмерный', 'pour', 'femme', '50', 'мл']],
 [4788809,
  ['сменный',
   'кассета',
   'мужской',
   'бритва',
   'gillette',
   'mach3',
   '3',
   'лезвие',
   'прочный',
   'сталь',
   'точный',
   'бритьё',
   '2',
   'шт']],
 [4789070,
  ['gillette',
   'fusion5',
   'мужской',
   'бритва',
   '2',
   'кассета',
   '5',
   'лезвие',
   'уменьшать',
   'трение',
   'точный',
   'триммер',
   'борода',
   'усов']],
 [5176579, ['lady', 'speed', 'stick', 'дезодорант']],
 [5501348,
  ['bourjois',
   'volume',
   'glamour',
   'тушь',
   'ресница',
   'объёмный',
   'чёрный',
   '12',
   'мл']],
 [5543514, ['wella', 'краска', 'волос', '110', 'мл']],
 [5629579,
  ['кислородный',
   'жидкий',
   'пятновыводитель',
   

#### Составление пар (token, doc_id)

In [74]:
corpus_tokens = [
    (token, doc_id) for doc_id, doc_tokens in sample_documents_processed for token in doc_tokens
]

In [75]:
corpus_tokens

[('модуль', 4036767),
 ('сменный', 4036767),
 ('фильтровать', 4036767),
 ('аквафора', 4036767),
 ('кн', 4036767),
 ('208731', 4036767),
 ('водоочиститель', 4050873),
 ('аквафора', 4050873),
 ('модель', 4050873),
 ('кристалл', 4050873),
 ('н', 4050873),
 ('205963', 4050873),
 ('кран', 4050873),
 ('развивать', 4226160),
 ('мышление', 4226160),
 ('2', 4226160),
 ('3', 4226160),
 ('год', 4226160),
 ('земцов', 4226160),
 ('ольга', 4226160),
 ('lacoste', 4644911),
 ('вода', 4644911),
 ('парфюмерный', 4644911),
 ('pour', 4644911),
 ('femme', 4644911),
 ('50', 4644911),
 ('мл', 4644911),
 ('сменный', 4788809),
 ('кассета', 4788809),
 ('мужской', 4788809),
 ('бритва', 4788809),
 ('gillette', 4788809),
 ('mach3', 4788809),
 ('3', 4788809),
 ('лезвие', 4788809),
 ('прочный', 4788809),
 ('сталь', 4788809),
 ('точный', 4788809),
 ('бритьё', 4788809),
 ('2', 4788809),
 ('шт', 4788809),
 ('gillette', 4789070),
 ('fusion5', 4789070),
 ('мужской', 4789070),
 ('бритва', 4789070),
 ('2', 4789070),
 ('кас

#### Сортировка по token, doc_id

In [76]:
corpus_tokens.sort()

In [77]:
corpus_tokens

[('110', 5543514),
 ('12', 5501348),
 ('2', 4226160),
 ('2', 4788809),
 ('2', 4789070),
 ('2', 5629579),
 ('205963', 4050873),
 ('208731', 4036767),
 ('3', 4226160),
 ('3', 4788809),
 ('5', 4789070),
 ('50', 4644911),
 ('action', 5629579),
 ('bourjois', 5501348),
 ('femme', 4644911),
 ('fusion5', 4789070),
 ('gillette', 4788809),
 ('gillette', 4789070),
 ('glamour', 5501348),
 ('lacoste', 4644911),
 ('lady', 5176579),
 ('mach3', 4788809),
 ('oxi', 5629579),
 ('pour', 4644911),
 ('speed', 5176579),
 ('stick', 5176579),
 ('vanish', 5629579),
 ('volume', 5501348),
 ('wella', 5543514),
 ('аквафора', 4036767),
 ('аквафора', 4050873),
 ('бельё', 5629579),
 ('борода', 4789070),
 ('бритва', 4788809),
 ('бритва', 4789070),
 ('бритьё', 4788809),
 ('ваниш', 5629579),
 ('вода', 4644911),
 ('водоочиститель', 4050873),
 ('волос', 5543514),
 ('год', 4226160),
 ('дезодорант', 5176579),
 ('жидкий', 5629579),
 ('земцов', 4226160),
 ('кассета', 4788809),
 ('кассета', 4789070),
 ('кислородный', 5629579),


#### Построение dictionary и posting lists

In [78]:
index = {}

for token, doc_id in corpus_tokens:
    if token in index:
        if doc_id == index[token][-1]:
            continue
        index[token].append(doc_id)
    else:
        index[token] = [doc_id]

In [79]:
index

{'110': [5543514],
 '12': [5501348],
 '2': [4226160, 4788809, 4789070, 5629579],
 '205963': [4050873],
 '208731': [4036767],
 '3': [4226160, 4788809],
 '5': [4789070],
 '50': [4644911],
 'action': [5629579],
 'bourjois': [5501348],
 'femme': [4644911],
 'fusion5': [4789070],
 'gillette': [4788809, 4789070],
 'glamour': [5501348],
 'lacoste': [4644911],
 'lady': [5176579],
 'mach3': [4788809],
 'oxi': [5629579],
 'pour': [4644911],
 'speed': [5176579],
 'stick': [5176579],
 'vanish': [5629579],
 'volume': [5501348],
 'wella': [5543514],
 'аквафора': [4036767, 4050873],
 'бельё': [5629579],
 'борода': [4789070],
 'бритва': [4788809, 4789070],
 'бритьё': [4788809],
 'ваниш': [5629579],
 'вода': [4644911],
 'водоочиститель': [4050873],
 'волос': [5543514],
 'год': [4226160],
 'дезодорант': [5176579],
 'жидкий': [5629579],
 'земцов': [4226160],
 'кассета': [4788809, 4789070],
 'кислородный': [5629579],
 'кн': [4036767],
 'кран': [4050873],
 'краска': [5543514],
 'кристалл': [4050873],
 'л':

In [80]:
def create_index(documents_processed: list[list[str]], limit: Optional[int] = None):
    sample_documents_processed = documents_processed[:limit]
    corpus_tokens = [
        (token, doc_id) for doc_id, doc_tokens in sample_documents_processed for token in doc_tokens
    ]
    corpus_tokens.sort()

    index = {}

    for token, doc_id in corpus_tokens:
        if token in index:
            if doc_id == index[token][-1]:
                continue
            index[token].append(doc_id)
        else:
            index[token] = [doc_id]

    return index


In [81]:
documents_index = create_index(documents_processed)

In [ ]:
with open("documents_index.json", "w") as f:
    json.dump(documents_index, f)


In [ ]:
with open("documents_index.json", "r") as f:
    documents_index = json.load(f)


## 3. Поиск по обратному индексу

In [83]:
def search_single_token_query(
    query: str, 
    index: dict[str, list[int]], 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer, 
    limit: Optional[int] = None,
) -> list[str]:
    query = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)[0]
    return [(result_doc_id, documents_dict[result_doc_id]) for result_doc_id in index[query]][:limit]

In [84]:
search_single_token_query(
    "диван",
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    10,
)

[(28876761,
  'Держатель пялец для вышивания на диване, кресле или за столом, Hobby Nurge'),
 (169810294,
  'Наматрасник 160х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 (169810305,
  'Наматрасник 80х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 (169810314,
  'Наматрасник 90х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 (170627411,
  'Наматрасник 180х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 (170627453,
  'Наматрасник 140х200 непромокаемый с р

### Максимизируем точность

#### Реализуем функцию, которая получает для каждого токена запроса список документов и реализует слияние списков через оператор AND

In [85]:
def merge_and(posting_lists: list[list[int]]):
    posting_lists.sort(key=len)
    result = posting_lists[0]

    for i in range(1, len(posting_lists)):
        current_list = posting_lists[i]
        new_result = []
        ptr1, ptr2 = 0, 0

        while ptr1 < len(result) and ptr2 < len(current_list):
            if result[ptr1] == current_list[ptr2]:
                new_result.append(result[ptr1])
                ptr1 += 1
                ptr2 += 1
            elif result[ptr1] < current_list[ptr2]:
                ptr1 += 1
            else:
                ptr2 += 1
        result = new_result

    return result
    

In [86]:
def search_with_and_condition(
    index: dict[str, list[int]], 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer,
    query: str, 
    limit: Optional[int] = None,
) -> list[str]:
    
    # 1. Обработка текста запроса
    # 2. Получение posting lists для каждого токена
    # 3. Слияние posting lists

    query_processed = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)

    merged_posting_lists = merge_and([index.get(token, []) for token in query_processed])

    return [
        Document(doc_id=doc_id, name=documents_dict[doc_id]) for doc_id in merged_posting_lists
    ]


In [87]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "диван кровать",
    10,
)

[Document(doc_id=177310072, name='Плед Павлайн Ирен какао 150х200 , плед на кровать; плед на диван; покрывало на кровать; покрывало на диван; плед 150 на 200; плед 200 на 220; 150*200; плед 200х220; плед 150х200; полуторка'),
 Document(doc_id=229424677, name='Пылесос для дома Jimmy WB55, ручной, для мебели и для удаления пылевых клещей, от шерсти животных, с контейнером, для диванов и кроватей, ультрафиолетовая и ультразвуковая чистка, мощный 600Вт'),
 Document(doc_id=229425205, name='Пылесос для дома Jimmy JV35, ручной, для мебели, для удаления пылевых клещей, от шерсти животных, с контейнером, для диванов и кроватей, ультрафиолетовая чистка и обработка горячим воздухом'),
 Document(doc_id=245377779, name='Покрывало на диван/кровать MIX 140х210 см'),
 Document(doc_id=255343578, name='Покрывало "Сладкий Сон" 220х240 см, двустороннее, однотонное, стеганое Ультрастеп. На кровать, на диван.'),
 Document(doc_id=262376994, name='Наматрасник стеганый 160 200 водонепроницаемый непромокаемый М

In [88]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "двухъярусная кровать",
    10,
)

[Document(doc_id=355105396, name='Двухъярусная кровать для кукол 46 см Our genration'),
 Document(doc_id=355115836, name="Набор мебели игровой Li'l Woodzeez Детская и двухъярусная кровать")]

In [89]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное",
    10,
)

[Document(doc_id=140987734, name='Valio Молоко Ультрапастеризованное 1.5% 250мл. 1шт.'),
 Document(doc_id=140987737, name='Valio Молоко Ультрапастеризованное 2.5% 1000мл. 1шт.'),
 Document(doc_id=140987742, name='Valio Молоко Ультрапастеризованное 3.5% 1000мл. 1шт.'),
 Document(doc_id=141165025, name='Молоко ультрапастеризованное Простоквашино, 3,2%, 950 мл'),
 Document(doc_id=141270536, name='Молоко Вкуснотеево ультрапастеризованное, 2,5%, 950 мл'),
 Document(doc_id=141270537, name='Молоко ультрапастеризованное, 3,2%, 950 мл, Вкуснотеево'),
 Document(doc_id=141580513, name='Молоко ультрапастеризованное 3,5%, 1 л, Parmalat'),
 Document(doc_id=141580514, name='Молоко ультрапастеризованное 1,8% 1 л, Parmalat'),
 Document(doc_id=141580515, name='Parmalat Молоко Ультрапастеризованное 0.5% 1000мл. 1шт.'),
 Document(doc_id=141580516, name='Parmalat Молоко Ультрапастеризованное 3.5% 200мл. 1шт.'),
 Document(doc_id=141580518, name='Молоко Белый Город ультрапастеризованное 3,2%, 1 л'),
 Documen

In [97]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное 1 л",
    10,
)

[Document(doc_id=141580513, name='Молоко ультрапастеризованное 3,5%, 1 л, Parmalat'),
 Document(doc_id=141580514, name='Молоко ультрапастеризованное 1,8% 1 л, Parmalat'),
 Document(doc_id=141580518, name='Молоко Белый Город ультрапастеризованное 3,2%, 1 л'),
 Document(doc_id=141876199, name='Молоко ультрапастеризованное 3,5%, 12 шт х 1 л, Parmalat'),
 Document(doc_id=141876200, name='Молоко ультрапастеризованное 1,8% 12 шт х 1 л, Parmalat'),
 Document(doc_id=141876201, name='Молоко Parmalat ультрапастеризованное 0,5%, 12 шт х 1 л'),
 Document(doc_id=143412236, name='Arla Natura Молоко безлактозное, ультрапастеризованное, 1,5%, 1 л'),
 Document(doc_id=156418189, name='Молоко ультрапастеризованное 3,5%, 12 шт х 1 л, Parmalat'),
 Document(doc_id=156418190, name='Молоко ультрапастеризованное 1,8% 12 шт х 1 л, Parmalat'),
 Document(doc_id=156418192, name='Молоко ультрапастеризованное 3,5%, 1 л, Parmalat'),
 Document(doc_id=156418193, name='Молоко Белый Город ультрапастеризованное 3,2%, 1 л'

In [91]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "для",
    10,
)

IndexError: list index out of range

In [92]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "средство для мытья посуды",
    10,
)

[Document(doc_id=6106583, name='Средство для мытья посуды "Frosch Лимон", 500 мл'),
 Document(doc_id=6106584, name='Средство для мытья посуды "Frosch", с ароматом лимона, 1 л'),
 Document(doc_id=23772174, name='LION Chamgreen Средство для мытья посуды, овощей и фруктов Зеленый чай 1200 мл'),
 Document(doc_id=26431422, name='Средство для мытья посуды, овощей и фруктов BioMio Bio-Care, гипоаллергенное, экологичное, без запаха, 450 мл'),
 Document(doc_id=26431424, name='Средство для мытья посуды, овощей и\xa0фруктов BioMio Bio-Care, гипоаллергенное, экологичное, с\xa0эфирным маслом мяты, 450\xa0мл'),
 Document(doc_id=26431428, name='Средство для мытья посуды, овощей и фруктов BioMio Bio-Care, гипоаллергенное, экологичное, с эфирным маслом мандарина, 450 мл'),
 Document(doc_id=29287948, name='LION Chamgreen Средство для мытья посуды, овощей и\xa0фруктов Японский абрикос, мягкая упаковка, 960\xa0мл'),
 Document(doc_id=33319465, name='Средство для мытья посуды SYNERGETIC антибактериальное,\x

In [93]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "скатерть на стол",
    10,
)

[Document(doc_id=148827514, name='Скатерть на стол 145х180 см из хлопка водоотталкивающая, скатерть кухонная на стол с пропиткой для праздника кухни дома сада и дачи пятнозащитная кухонный декор в подарок, белая с розами прованс'),
 Document(doc_id=148827515, name='Скатерть на стол Fresca Design 145х220 см'),
 Document(doc_id=149750466, name='Скатерть круглая 185 см на стол из хлопка водоотталкивающая, скатерть на круглый стол кухонная с пропиткой от воды для праздника кухни дома сада и дачи пятнозащитная кухонный текстиль декор в подарок'),
 Document(doc_id=151998971, name='Скатерть на стол UNTERZO home прямоугольная 145х180 см хлопок'),
 Document(doc_id=155432339, name='Скатерть на стол круглая Fresca Design 185х185 см хлопок'),
 Document(doc_id=155523748, name='Скатерть на стол овальная 160х220 см хлопок не боится пятен'),
 Document(doc_id=155523854, name='Скатерть на стол Fresca Design 145х220 см'),
 Document(doc_id=155523861, name='Скатерть на стол квадратная 145 см хлопок'),
 Doc

In [94]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "скатерть для стола",
    10,
)

[Document(doc_id=148827514, name='Скатерть на стол 145х180 см из хлопка водоотталкивающая, скатерть кухонная на стол с пропиткой для праздника кухни дома сада и дачи пятнозащитная кухонный декор в подарок, белая с розами прованс'),
 Document(doc_id=148827515, name='Скатерть на стол Fresca Design 145х220 см'),
 Document(doc_id=149750466, name='Скатерть круглая 185 см на стол из хлопка водоотталкивающая, скатерть на круглый стол кухонная с пропиткой от воды для праздника кухни дома сада и дачи пятнозащитная кухонный текстиль декор в подарок'),
 Document(doc_id=151998971, name='Скатерть на стол UNTERZO home прямоугольная 145х180 см хлопок'),
 Document(doc_id=155432339, name='Скатерть на стол круглая Fresca Design 185х185 см хлопок'),
 Document(doc_id=155523748, name='Скатерть на стол овальная 160х220 см хлопок не боится пятен'),
 Document(doc_id=155523854, name='Скатерть на стол Fresca Design 145х220 см'),
 Document(doc_id=155523861, name='Скатерть на стол квадратная 145 см хлопок'),
 Doc

In [95]:
calculate_validation_metrics(
    partial(
        search_with_and_condition,
        documents_index,
        documents_dict,
        symbols_to_replace,
        russian_stopwords,
        morph,
    ),
    limit=20
)

precision = 0.09579784216969496
recall = 0.2862654838254138
f1_score = 0.0971321719689694

### Максимизируем полноту

#### Реализуем функцию, которая получает для каждого токена запроса список документов и реализует слияние списков через оператор OR

In [98]:
def merge_or(posting_lists: list[list[int]]):
    result = set()
    for lst in posting_lists:
        result.update(lst)

    return result


In [99]:
def search_with_or_condition(
    index: dict[str, list[int]], 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer, 
    query: str, 
    limit: Optional[int] = None,
) -> list[str]:
    
    # 1. Обработка запроса
    # 2. Получение posting lists для каждого токена
    # 3. Слияние posting lists
    
    query_processed = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)

    merged_posting_lists = merge_or([index.get(token, []) for token in query_processed])

    return [
        Document(doc_id=doc_id, name=documents_dict[doc_id]) for doc_id in merged_posting_lists
    ]


In [100]:
search_with_or_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное",
    10,
)

[Document(doc_id=1442791428, name='Parmalat Молоко Ультрапастеризованное 3.5% 1000мл. 12шт. Уцененный товар'),
 Document(doc_id=1558601740, name='Село Зеленое Молоко питьевое ультрапастеризованное 3,2%, 950 мл х 12 шт. Уцененный товар'),
 Document(doc_id=148054037, name='Zeitun Скраб для тела с маслами, питательный, сахарный "Шоколад и молоко" 250 мл'),
 Document(doc_id=1593598002, name='Молочная смесь Nestle NAN с рождения до 12 месяцев, на козьем молоке, 400 г. Уцененный товар'),
 Document(doc_id=175415348, name='Молоко цельное сгущенное с сахаром, Рогачевъ, ГОСТ, 8,5 %, дой-пак, 270 гр., 3 штуки'),
 Document(doc_id=1525588024, name='Молоко Стерилизованное Село Зеленое 2,5 % 0,95 л х 12 шт. Уцененный товар'),
 Document(doc_id=472080455, name='Сок с молоком Мажитэль Папайя-Манго-Ананас 0,05%, 950 г'),
 Document(doc_id=231555166, name='Молочко детское Kabrita Gold 4, с 18 месяцев, на козьем молоке для комфортного пищеварения, 800 г'),
 Document(doc_id=1439965285, name='Parmalat Молоко 

In [101]:
search_with_or_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное 1литр",
    10,
)

[Document(doc_id=1442791428, name='Parmalat Молоко Ультрапастеризованное 3.5% 1000мл. 12шт. Уцененный товар'),
 Document(doc_id=1558601740, name='Село Зеленое Молоко питьевое ультрапастеризованное 3,2%, 950 мл х 12 шт. Уцененный товар'),
 Document(doc_id=148054037, name='Zeitun Скраб для тела с маслами, питательный, сахарный "Шоколад и молоко" 250 мл'),
 Document(doc_id=1593598002, name='Молочная смесь Nestle NAN с рождения до 12 месяцев, на козьем молоке, 400 г. Уцененный товар'),
 Document(doc_id=175415348, name='Молоко цельное сгущенное с сахаром, Рогачевъ, ГОСТ, 8,5 %, дой-пак, 270 гр., 3 штуки'),
 Document(doc_id=1525588024, name='Молоко Стерилизованное Село Зеленое 2,5 % 0,95 л х 12 шт. Уцененный товар'),
 Document(doc_id=472080455, name='Сок с молоком Мажитэль Папайя-Манго-Ананас 0,05%, 950 г'),
 Document(doc_id=231555166, name='Молочко детское Kabrita Gold 4, с 18 месяцев, на козьем молоке для комфортного пищеварения, 800 г'),
 Document(doc_id=1439965285, name='Parmalat Молоко 

In [102]:
search_with_or_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "диван кровать",
    10,
)

[Document(doc_id=1489766400, name='Средство пятновыводитель для чистки ковров, дивана,салона и удаления пятен на мягкой мебели WONDER LAB, без запаха, 550 мл. Уцененный товар'),
 Document(doc_id=282235399, name='Накладка на подлокотник дивана DECOREZ / Подставка деревянная 46*29 см'),
 Document(doc_id=258876942, name='Накладка на подлокотник дивана DECOREZ / Подставка деревянная 46*29 см'),
 Document(doc_id=270393871, name='Защитные накладка на стул, 22мм. 24 штуки. Наклейка войлочная на ножку мебели, подпятник на диван, насадки мебельные'),
 Document(doc_id=302585365, name='Наматрасник 160х200 непромокаемый c бортами на резинке, чехол на матрас или диван аквастоп, простынь непромокаемая водонепроницаемая на резинке, топпер, наматрацник'),
 Document(doc_id=462384665, name='CINLANKIDS Защитный бортик для детской кровати 180*98 см от падения, серый (высота регулируется), 1 шт.'),
 Document(doc_id=262376994, name='Наматрасник стеганый 160 200 водонепроницаемый непромокаемый Мягкость&Комфо

In [103]:
calculate_validation_metrics(
    partial(
        search_with_or_condition,
        documents_index,
        documents_dict,
        symbols_to_replace,
        russian_stopwords,
        morph,
    ),
    limit=20,
)

precision = 0.038091081542604974
recall = 0.6583476100097401
f1_score = 0.05104573014970413

## 3.1 Введение в ранжирование

#### В обратный индекс можно добавить term frequency

In [104]:
def create_index_with_tf(documents_processed: list[list[str]], limit: Optional[int] = None):
    sample_documents_processed = documents_processed[:limit]
    corpus_tokens = [
        (token, doc_id) for doc_id, doc_tokens in sample_documents_processed for token in doc_tokens
    ]
    corpus_tokens.sort()

    index = {}

    for token, doc_id in corpus_tokens:
        if token in index:
            if doc_id == index[token][-1][0]:
                index[token][-1][1] += 1
                continue
            index[token].append([doc_id, 1])
        else:
            index[token] = [[doc_id, 1]]

    return index


In [105]:
index_with_tf = create_index_with_tf(documents_processed)

In [106]:
index_with_tf["диван"]

[[28876761, 1],
 [169810294, 1],
 [169810305, 1],
 [169810314, 1],
 [170627411, 1],
 [170627453, 1],
 [170627456, 1],
 [172686265, 1],
 [172686268, 1],
 [172686269, 1],
 [172686270, 1],
 [174148448, 1],
 [174443280, 1],
 [174443281, 1],
 [174443283, 1],
 [177310072, 2],
 [203595521, 1],
 [203598178, 1],
 [204416930, 1],
 [229424677, 1],
 [229425205, 1],
 [245377779, 1],
 [255343578, 1],
 [258876942, 1],
 [258901916, 1],
 [258908855, 1],
 [259150828, 1],
 [262376994, 1],
 [270393871, 1],
 [279745854, 1],
 [282182889, 1],
 [282235399, 1],
 [282764375, 1],
 [302585365, 1],
 [311565940, 1],
 [325179712, 1],
 [339324277, 1],
 [344386617, 2],
 [361843781, 1],
 [367354212, 1],
 [367555536, 1],
 [367575426, 1],
 [367579588, 1],
 [383425290, 1],
 [414307639, 1],
 [418312504, 1],
 [421452489, 1],
 [424884701, 1],
 [478237691, 1],
 [495117560, 2],
 [496408166, 2],
 [509211202, 1],
 [514356916, 1],
 [522969042, 1],
 [523819944, 1],
 [525483453, 1],
 [541335282, 1],
 [546303575, 1],
 [563247577, 1]

In [107]:
index_with_tf["молоко"]

[[31413018, 1],
 [33705241, 1],
 [135456706, 1],
 [135789425, 1],
 [135789426, 1],
 [136780595, 1],
 [136780600, 1],
 [137673490, 1],
 [137673491, 1],
 [137673492, 1],
 [138133907, 1],
 [140463353, 1],
 [140475265, 1],
 [140475273, 1],
 [140675230, 1],
 [140675231, 1],
 [140675232, 1],
 [140675233, 1],
 [140675234, 1],
 [140676398, 1],
 [140676409, 1],
 [140676410, 1],
 [140676411, 1],
 [140676412, 1],
 [140987734, 1],
 [140987736, 1],
 [140987737, 1],
 [140987738, 1],
 [140987742, 1],
 [140987743, 1],
 [141133760, 2],
 [141165025, 1],
 [141165031, 1],
 [141165033, 1],
 [141270536, 1],
 [141270537, 1],
 [141544350, 1],
 [141580513, 1],
 [141580514, 1],
 [141580515, 1],
 [141580516, 1],
 [141580518, 1],
 [141580519, 1],
 [141580520, 1],
 [141580524, 1],
 [141876199, 1],
 [141876200, 1],
 [141876201, 1],
 [141927409, 1],
 [142120587, 1],
 [142120588, 1],
 [142120589, 1],
 [142120590, 1],
 [142479173, 1],
 [142624198, 1],
 [142731191, 1],
 [142731192, 1],
 [143321964, 1],
 [143321967, 1],

In [ ]:
with open("documents_index_with_tf.json", "w") as f:
    json.dump(index_with_tf, f)


In [111]:
with open("documents_index_with_tf.json", "r") as f:
    documents_index_with_tf = json.load(f)

### Взвешенный поиск по индексу

In [112]:
def search_with_tf_with_or_condition(
    index: dict[str, list[int]], 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer, 
    query: str, 
    limit: Optional[int] = None,
) -> list[str]:
    query_processed = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)

    result = dict()
    for token in query_processed:
        token_results = index.get(token, [])
        for token_result in token_results:
            doc_id, tf = token_result[0], token_result[1]
            if doc_id in result:
                result[doc_id] += tf
            else:
                result[doc_id] = tf

    return sorted(
        [Document(doc_id=result_doc_id, name=[documents_dict[result_doc_id], score]) for result_doc_id, score in result.items()],
        key=lambda x: -x.name[1]
    )[:limit]


In [113]:
search_with_tf_with_or_condition(
    documents_index_with_tf,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "диван кровать",
    10,
)

[Document(doc_id=177310072, name=['Плед Павлайн Ирен какао 150х200 , плед на кровать; плед на диван; покрывало на кровать; покрывало на диван; плед 150 на 200; плед 200 на 220; 150*200; плед 200х220; плед 150х200; полуторка', 4]),
 Document(doc_id=496408166, name=['Покрывало на кровать и диван 117х200 см стеганое, плед для дивана детской и спальни в подарок, бархат, серое АМИ МЕБЕЛЬ Беларусь', 3]),
 Document(doc_id=567905258, name=['Покрывало на кровать и диван 140х220 см стеганое 1,5 спальное, плед для дивана детской и спальни в подарок, бархат, АМИ МЕБЕЛЬ Беларусь', 3]),
 Document(doc_id=627844666, name=['Покрывало плед шерстяной 200х220 см PLEDICO "Скандинавский стиль" на кровать диван евро, серый белый, накидка на диван шерсть, подарочная упаковка, производство Россия', 3]),
 Document(doc_id=655020385, name=['Плед 200х220 на кровать LUZIA, покрывало на кровать, для дивана, двухсторонний, 60% хлопок, бежевый', 3]),
 Document(doc_id=760196461, name=['Плед 200х220 для дивана LAVAL, по

In [114]:
search_with_tf_with_or_condition(
    documents_index_with_tf,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное 1литр",
    10,
)

[Document(doc_id=1615880294, name=['Молоко ультрапастеризованное Просто Молоко, 3,2 %, 1 л', 3]),
 Document(doc_id=1615880546, name=['Молоко ультрапастеризованное Просто Молоко, 2,5 %, 1 л', 3]),
 Document(doc_id=140987734, name=['Valio Молоко Ультрапастеризованное 1.5% 250мл. 1шт.', 2]),
 Document(doc_id=140987737, name=['Valio Молоко Ультрапастеризованное 2.5% 1000мл. 1шт.', 2]),
 Document(doc_id=140987742, name=['Valio Молоко Ультрапастеризованное 3.5% 1000мл. 1шт.', 2]),
 Document(doc_id=141133760, name=['Шоколад Ritter Sport "Альпийское молоко" с альпийским молоком, 100 г', 2]),
 Document(doc_id=141165025, name=['Молоко ультрапастеризованное Простоквашино, 3,2%, 950 мл', 2]),
 Document(doc_id=141270536, name=['Молоко Вкуснотеево ультрапастеризованное, 2,5%, 950 мл', 2]),
 Document(doc_id=141270537, name=['Молоко ультрапастеризованное, 3,2%, 950 мл, Вкуснотеево', 2]),
 Document(doc_id=141580513, name=['Молоко ультрапастеризованное 3,5%, 1 л, Parmalat', 2])]

In [115]:
calculate_validation_metrics(
    partial(
        search_with_tf_with_or_condition,
        documents_index_with_tf,
        documents_dict,
        symbols_to_replace,
        russian_stopwords,
        morph,
    ),
    limit=20,
)

precision = 0.11266666666666668
recall = 0.11471620251678676
f1_score = 0.10590267610832985

### Альтернативное хранение словаря - префиксное дерево поиска aka trie

In [121]:
class TrieNode:
    def __init__(self):
        self.children: dict[str, "TrieNode"] = {}
        self.is_end_of_word: bool = False
        self.document_ids: list[int] = set()

class TrieInvertedIndex:
    def __init__(self):
        self.root = TrieNode()

    def _insert_document(self, token: str, doc_id: int) -> None:
        node = self.root

        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]

        node.is_end_of_word = True
        node.document_ids.add(doc_id)
    
    def insert_document(self, doc_id: int, tokens: list[str]) -> None:
        for token in tokens:
            self._insert_document(token, doc_id)

    def _search_node(self, token: str) -> TrieNode:
        node = self.root
        
        for char in token:
            if char not in node.children:
                return None
            node = node.children[char]

        return node
    
    def search(self, token: str) -> list[int]:
        node = self._search_node(token)

        if node and node.is_end_of_word:
            return sorted(node.document_ids)
        return []


In [122]:
def create_trie_index(documents_processed: list[tuple[int, list[str]]], limit: Optional[int] = None) -> TrieInvertedIndex:
    sample_documents_processed = documents_processed[:limit] if limit else documents_processed
    index = TrieInvertedIndex()
    
    for doc_id, tokens in sample_documents_processed:
        index.insert_document(doc_id, tokens)
    
    return index

In [123]:
trie_index = create_trie_index(documents_processed)

In [124]:
def search_with_and_condition(
    index: TrieInvertedIndex, 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer,
    query: str, 
    limit: Optional[int] = None,
) -> list[str]:
    query_processed = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)
    if not query_processed:
        return []
    merged_posting_lists = merge_and(
        [index.search(token) for token in query_processed]
    )
    
    return [
        Document(doc_id=result_doc_id, name=documents_dict[result_doc_id]) for result_doc_id in merged_posting_lists
    ][:limit]

In [125]:
search_with_and_condition(
    trie_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "двухъярусная кровать",
    10,
)

[Document(doc_id=355105396, name='Двухъярусная кровать для кукол 46 см Our genration'),
 Document(doc_id=355115836, name="Набор мебели игровой Li'l Woodzeez Детская и двухъярусная кровать")]

In [126]:
search_with_and_condition(
    trie_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное",
    10,
)

[Document(doc_id=140987734, name='Valio Молоко Ультрапастеризованное 1.5% 250мл. 1шт.'),
 Document(doc_id=140987737, name='Valio Молоко Ультрапастеризованное 2.5% 1000мл. 1шт.'),
 Document(doc_id=140987742, name='Valio Молоко Ультрапастеризованное 3.5% 1000мл. 1шт.'),
 Document(doc_id=141165025, name='Молоко ультрапастеризованное Простоквашино, 3,2%, 950 мл'),
 Document(doc_id=141270536, name='Молоко Вкуснотеево ультрапастеризованное, 2,5%, 950 мл'),
 Document(doc_id=141270537, name='Молоко ультрапастеризованное, 3,2%, 950 мл, Вкуснотеево'),
 Document(doc_id=141580513, name='Молоко ультрапастеризованное 3,5%, 1 л, Parmalat'),
 Document(doc_id=141580514, name='Молоко ультрапастеризованное 1,8% 1 л, Parmalat'),
 Document(doc_id=141580515, name='Parmalat Молоко Ультрапастеризованное 0.5% 1000мл. 1шт.'),
 Document(doc_id=141580516, name='Parmalat Молоко Ультрапастеризованное 3.5% 200мл. 1шт.')]

#### Что получили:

1. Экономия памяти для схожих токенов

2. Эффективный поиск по префиксу - можно находить все токены, начинающиеся с определенной последовательности символов